In [ ]:
# ✅ Instalação de dependências (apenas uma vez)
%pip install -U transformers datasets accelerate bitsandbytes peft trl

In [ ]:
# ✅ Carregar dataset supervisionado
from datasets import load_dataset

dataset = load_dataset("json", data_files="/mnt/data/dataset_operacoes_supervisionado.jsonl", split="train")
dataset = dataset.train_test_split(test_size=0.2)
dataset

In [ ]:
# ✅ Preparar o modelo base para fine-tuning (exemplo com Mistral)
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-v0.1"  # Pode substituir por outro modelo local ou pequeno
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

In [ ]:
# ✅ Tokenizar o dataset
def tokenize(example):
    prompt = example["prompt"]
    response = example["response"]
    full_text = f"{prompt}\n{response}"
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize)
tokenized_dataset

In [ ]:
# ✅ Aplicar LoRA para fine-tuning leve
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# ✅ Treinamento do modelo com Trainer
training_args = TrainingArguments(
    output_dir="./modelo_treinado",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=50,
    evaluation_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

trainer.train()

In [ ]:
# ✅ Salvar o modelo fine-tuned
model.save_pretrained("modelo_final_priceaction")
tokenizer.save_pretrained("modelo_final_priceaction")
print("✅ Modelo fine-tuned salvo com sucesso!")